# Project 2 - Lab 5 - Filter and aggregate the water quality data

Recall that one of the files (starts with `mces`) contains water quality measurements for lakes in the Twin Cities.  In this lab, we will narrow down the list of lakes for which we have at least one of each measurement type (phosphorus and secchi depth) for each year between 2004 and 2015.

**Important note.** Recall that we fixed an issue with the water quality data in our terminal exploration of the data. Be sure to use the corrected version of the water quality data located in the `data/MinneMUDAC_raw_files_fixed/mces_lakes_1999_2014_v2.txt` file.

## Problem 1 - Inspect the data

We will be focusing on two of water quality measurements: phosphorus and secchi depth.  Before we start trimming the data set, we should explore these metrics.

1. Each of the measures has a `QUALIFIER` column.  Group and aggregate by each of these columns and note any problematic values.  **Hint.** This search should indicate that some of the phosphorus measurements should be dropped.  Make sure you include this action as part of your primary query.
2. Each measure also includes a `Units` column.  Check that all measurement are in the same units, and convert as needed.

In [37]:
import polars as pl
import polars.selectors as cs
from glob import glob
import re
from humanize import naturalsize
import os
from operator import mul

In [3]:
glob('./data/MinneMUDAC_raw_files/*.txt')

['./data/MinneMUDAC_raw_files\\2002_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\2003_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\2004_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\2005_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\2006_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\2007_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\2008_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\2009_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\2010_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\2011_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\2012_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\2013_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\2014_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\2015_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files\\mces_lakes_1999_2014.txt',
 './data/MinneMUDAC_raw_files\\mces_lakes_1999_2014_v2.txt',
 './data/MinneMUDAC_raw_files\\Parcel_Lak

In [4]:
(water_quality := pl.read_csv('./data/MinneMUDAC_raw_files/mces_lakes_1999_2014_v2.txt',
                              separator='\t',
                              infer_schema_length=10000)
)

PROJECT_ID,DATA_SET_TITLE,LAKE_NAME,CITY,COUNTY,DNR_ID_Site_Number,MAJOR_WATERSHED,WATER_PLANNING_AUTHORITY,LAKE_SITE_NUMBER,START_DATE,START_HOURMIN24,END_DATE,END_HOURMIN24,SAMPLE_DEPTH_IN_METERS,Seasonal_Lake_Grade_RESULT,Seasonal_Lake_Grade_QUALIFIER,Seasonal_Lake_Grade_Units,Physical_Condition_RESULT,Physical_Condition_QUALIFIER,Physical_Condition_Units,Recreational_Suitability_RESULT,Recreational_Suitability_QUALIFIER,Recreational_Suitability_Units,Secchi_Depth_RESULT_SIGN,Secchi_Depth_RESULT,Secchi_Depth_QUALIFIER,Secchi_Depth_Units,Total_Phosphorus_RESULT_SIGN,Total_Phosphorus_RESULT,Total_Phosphorus_QUALIFIER,Total_Phosphorus_Units,longitude,latitude
i64,str,str,str,str,str,str,str,i64,str,str,str,str,f64,i64,str,str,i64,str,str,i64,str,str,str,f64,str,str,str,f64,str,str,f64,f64
7108,"""Citizen Assisted Monitoring Pr…","""Acorn Lake""","""Oakdale""","""Washington""","""82010200-01""","""Lower St. Croix River""","""Valley Branch WD""",1,"""2006-04-16""","""0:00""","""2006-04-16""","""0:00""",0.0,null,null,"""0-4 Categorical Calculated Sea…",1,"""Approved""","""1-5 Categorical: 1 good & 5 ba…",5,"""Approved""","""1-5 Categorical: 1 good & 5 ba…",null,1.0,"""Approved""","""m""",null,0.156,"""Approved""","""mg/L""",-92.971711,45.016556
7108,"""Citizen Assisted Monitoring Pr…","""Acorn Lake""","""Oakdale""","""Washington""","""82010200-01""","""Lower St. Croix River""","""Valley Branch WD""",1,"""2006-05-01""","""0:00""","""2006-09-30""","""0:00""",0.0,2,"""Approved""","""0-4 Categorical Calculated Sea…",null,null,"""1-5 Categorical: 1 good & 5 ba…",null,null,"""1-5 Categorical: 1 good & 5 ba…",null,null,null,"""m""",null,null,null,"""mg/L""",-92.971711,45.016556
7108,"""Citizen Assisted Monitoring Pr…","""Acorn Lake""","""Oakdale""","""Washington""","""82010200-01""","""Lower St. Croix River""","""Valley Branch WD""",1,"""2006-05-02""","""0:00""","""2006-05-02""","""0:00""",0.0,null,null,"""0-4 Categorical Calculated Sea…",1,"""Approved""","""1-5 Categorical: 1 good & 5 ba…",5,"""Approved""","""1-5 Categorical: 1 good & 5 ba…",null,0.66,"""Approved""","""m""",null,0.107,"""Approved""","""mg/L""",-92.971711,45.016556
7108,"""Citizen Assisted Monitoring Pr…","""Acorn Lake""","""Oakdale""","""Washington""","""82010200-01""","""Lower St. Croix River""","""Valley Branch WD""",1,"""2006-05-16""","""0:00""","""2006-05-16""","""0:00""",0.0,null,null,"""0-4 Categorical Calculated Sea…",2,"""Approved""","""1-5 Categorical: 1 good & 5 ba…",5,"""Approved""","""1-5 Categorical: 1 good & 5 ba…",null,0.66,"""Approved""","""m""",null,0.141,"""Approved""","""mg/L""",-92.971711,45.016556
7108,"""Citizen Assisted Monitoring Pr…","""Acorn Lake""","""Oakdale""","""Washington""","""82010200-01""","""Lower St. Croix River""","""Valley Branch WD""",1,"""2006-05-30""","""0:00""","""2006-05-30""","""0:00""",0.0,null,null,"""0-4 Categorical Calculated Sea…",2,"""Approved""","""1-5 Categorical: 1 good & 5 ba…",5,"""Approved""","""1-5 Categorical: 1 good & 5 ba…",null,0.5,"""Approved""","""m""",null,0.029,"""Approved""","""mg/L""",-92.971711,45.016556
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
7105,"""Lakes Monitoring""","""Zumbra Lake""","""Victoria""","""Carver""","""10004100-01""","""Mississippi River - Twin Citie…","""Minnehaha Creek WD""",1,"""2002-09-16""","""0:00""","""2002-09-16""","""0:00""",10.0,null,null,null,null,null,null,null,null,null,null,null,null,"""m""",null,0.224,"""Approved""","""mg/L""",-93.667689,44.883817
7105,"""Lakes Monitoring""","""Zumbra Lake""","""Victoria""","""Carver""","""10004100-01""","""Mississippi River - Twin Citie…","""Minnehaha Creek WD""",1,"""2002-10-01""","""0:00""","""2002-10-01""","""0:00""",0.0,null,null,null,null,null,null,null,null,null,null,2.7,"""Approved""","""m""","""~""",0.026,"""Approved""","""mg/L""",-93.667689,44.883817
7105,"""Lakes Monitoring""","""Zumbra Lake""","""Victoria""","""Carver""","""10004100-01""","""Mississippi River - Twin Citie…","""Minnehaha Creek WD""",1

In [5]:
water_quality.columns

['PROJECT_ID',
 'DATA_SET_TITLE',
 'LAKE_NAME',
 'CITY',
 'COUNTY',
 'DNR_ID_Site_Number',
 'MAJOR_WATERSHED',
 'WATER_PLANNING_AUTHORITY',
 'LAKE_SITE_NUMBER',
 'START_DATE',
 'START_HOURMIN24',
 'END_DATE',
 'END_HOURMIN24',
 'SAMPLE_DEPTH_IN_METERS',
 'Seasonal_Lake_Grade_RESULT',
 'Seasonal_Lake_Grade_QUALIFIER',
 'Seasonal_Lake_Grade_Units',
 'Physical_Condition_RESULT',
 'Physical_Condition_QUALIFIER',
 'Physical_Condition_Units',
 'Recreational_Suitability_RESULT',
 'Recreational_Suitability_QUALIFIER',
 'Recreational_Suitability_Units',
 'Secchi_Depth_RESULT_SIGN',
 'Secchi_Depth_RESULT',
 'Secchi_Depth_QUALIFIER',
 'Secchi_Depth_Units',
 'Total_Phosphorus_RESULT_SIGN',
 'Total_Phosphorus_RESULT',
 'Total_Phosphorus_QUALIFIER',
 'Total_Phosphorus_Units',
 'longitude',
 'latitude']

In [28]:
wq_cols_to_keep = ['DNR_ID_Site_Number',
                   'END_DATE',
                   'LAKE_NAME',
                   'Secchi_Depth_RESULT_SIGN',
                   'Secchi_Depth_RESULT',
                   'Secchi_Depth_QUALIFIER',
                   'Secchi_Depth_Units',
                   'Total_Phosphorus_RESULT_SIGN',
                   'Total_Phosphorus_RESULT',
                   'Total_Phosphorus_QUALIFIER',
                   'Total_Phosphorus_Units',
                   'longitude',
                   'latitude'
                   ]

In [9]:
(water_quality
 .group_by('Secchi_Depth_QUALIFIER')
 .len()
)

Secchi_Depth_QUALIFIER,len
str,u32
"""Approved""",35104
null,13153


In [10]:
(water_quality
 .group_by('Total_Phosphorus_QUALIFIER')
 .len()
)

Total_Phosphorus_QUALIFIER,len
str,u32
"""Approved""",43639
"""Suspect""",35
null,4583


In [11]:
(water_quality
 .group_by('Secchi_Depth_Units')
 .len()
)

Secchi_Depth_Units,len
str,u32
"""m""",48257


## Problem 2 - Find filter and aggregate.

#### Tasks

Remember that our goal is to narrow the data to one row per lake per year.  Build a query that groups and aggregates to find the yearly average values for both phosphorus and secchi depth.  To do this your will want to

1. Filter based on what you learned in **Problem 1.**
2. Make sure that the `END_DATE` has the correct type and extract the year.  
3. Filter to the correct range of years.
4. Now you should group and aggregate to compute the yearly means.  We want to keep both the `LAKE_NAME` and lake ID to allow us to join these data to the parcel features we will construct in the next lab.

In [29]:
(water_quality_filtered := 
 pl.read_csv('./data/MinneMUDAC_raw_files/mces_lakes_1999_2014_v2.txt',
                              separator='\t',
                              infer_schema_length=10000,
                              columns=wq_cols_to_keep)
   .filter((pl.col('Secchi_Depth_QUALIFIER') == 'Approved') & 
           (pl.col('Total_Phosphorus_QUALIFIER') == 'Approved'))
   .with_columns(pl.col('END_DATE').str.split('-').list.get(0).cast(pl.Int32).alias('Year'))
   .filter((pl.col('Year') >= 2004) & (pl.col('Year') <= 2015))
 )

LAKE_NAME,DNR_ID_Site_Number,END_DATE,Secchi_Depth_RESULT_SIGN,Secchi_Depth_RESULT,Secchi_Depth_QUALIFIER,Secchi_Depth_Units,Total_Phosphorus_RESULT_SIGN,Total_Phosphorus_RESULT,Total_Phosphorus_QUALIFIER,Total_Phosphorus_Units,longitude,latitude,Year
str,str,str,str,f64,str,str,str,f64,str,str,f64,f64,i32
"""Acorn Lake""","""82010200-01""","""2006-04-16""",null,1.0,"""Approved""","""m""",null,0.156,"""Approved""","""mg/L""",-92.971711,45.016556,2006
"""Acorn Lake""","""82010200-01""","""2006-05-02""",null,0.66,"""Approved""","""m""",null,0.107,"""Approved""","""mg/L""",-92.971711,45.016556,2006
"""Acorn Lake""","""82010200-01""","""2006-05-16""",null,0.66,"""Approved""","""m""",null,0.141,"""Approved""","""mg/L""",-92.971711,45.016556,2006
"""Acorn Lake""","""82010200-01""","""2006-05-30""",null,0.5,"""Approved""","""m""",null,0.029,"""Approved""","""mg/L""",-92.971711,45.016556,2006
"""Acorn Lake""","""82010200-01""","""2006-06-11""",null,0.5,"""Approved""","""m""",null,0.058,"""Approved""","""mg/L""",-92.971711,45.016556,2006
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Woodpile Lake""","""82013200-01""","""2014-08-13""",null,3.05,"""Approved""","""m""",null,0.023,"""Approved""","""mg/L""",-92.903328,45.068687,2014
"""Woodpile Lake""","""82013200-01""","""2014-08-25""",null,2.74,"""Approved""","""m""",null,0.026,"""Approved""","""mg/L""",-92.903328,45.068687,2014
"""Woodpile Lake""","""82013200-01""","""2014-09-09""",null,2.13,"""Approved""","""m""",null,0.03,"""Approved""","""mg/L""",-92.903328,45.068687,2014


In [30]:
(water_quality_summaries := water_quality_filtered
 .group_by(['DNR_ID_Site_Number', 'Year', 'LAKE_NAME'])
 .agg([pl.col('Secchi_Depth_RESULT').mean().alias('avg_secchi_depth'),
       pl.col('Total_Phosphorus_RESULT').mean().alias('avg_total_phosphorus')])
 )

DNR_ID_Site_Number,Year,LAKE_NAME,avg_secchi_depth,avg_total_phosphorus
str,i32,str,f64,f64
"""02002200-01""",2007,"""Island Lake""",1.576923,0.025923
"""82015300-01""",2012,"""Sunset Lake""",3.164286,0.025429
"""13004200-01""",2005,"""Birch Lake""",1.513214,0.103714
"""27071100-01""",2010,"""Westwood Lake""",1.327778,0.022667
"""02058500-01""",2013,"""Lochness Lake""",2.027778,0.033222
…,…,…,…,…
"""10000900-02""",2013,"""Minnewashta Lake""",2.37,0.0175
"""82046200-01""",2009,"""Legion Pond""",0.381,0.058
"""82013200-01""",2006,"""Woodpile Lake""",1.349714,0.074143


## Problem 3 - Find lakes with complete yearly averages.

We want to make sure that we don't have any missing data in the target vectors, so we need to build a query that leads to a list of lake names and codes that fit the following criteria.

1. Only contains years after 2003.
2. Has a non-missing value for both means.
3. Contains both the lake name and the lake code.

You should save this list of lake IDs in a variable named `lakes_w_complete_info` in a file named `lake.py`.  Restart your kernel and confirm that you can import this data.

In [41]:
(wq_complete_information := water_quality_summaries
 .with_columns((pl.col('avg_secchi_depth').is_not_null() & pl.col('avg_total_phosphorus').is_not_null())
                .cast(pl.Int8)
                .alias('summary_complete'))
 .pivot(values='summary_complete',
        index=['DNR_ID_Site_Number', 'LAKE_NAME'],
        on='Year',
        aggregate_function='sum'
        )
 .with_columns(all_complete = pl.reduce(mul, cs.integer()))
 .filter(pl.col('all_complete') == 1)
 )

DNR_ID_Site_Number,LAKE_NAME,2007,2012,2005,2010,2013,2006,2009,2011,2008,2004,2014,all_complete
str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""82015300-01""","""Sunset Lake""",1,1,1,1,1,1,1,1,1,1,1,1
"""27071100-01""","""Westwood Lake""",1,1,1,1,1,1,1,1,1,1,1,1
"""82033400-01""","""Kismet Lake""",1,1,1,1,1,1,1,1,1,1,1,1
"""82011301-01""","""Goose Lake""",1,1,1,1,1,1,1,1,1,1,1,1
"""82008700-01""","""Regional Park Lake""",1,1,1,1,1,1,1,1,1,1,1,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""82002000-01""","""McKusick Lake""",1,1,1,1,1,1,1,1,1,1,1,1
"""19003100-01""","""Orchard Lake""",1,1,1,1,1,1,1,1,1,1,1,1
"""19002500-01""","""Keller Lake""",1,1,1,1,1,1,1,1,1,1,1,1


In [42]:
(dnr_id_site_numbers := wq_complete_information
 .get_column('DNR_ID_Site_Number')
 .to_list()
)

['82015300-01',
 '27071100-01',
 '82033400-01',
 '82011301-01',
 '82008700-01',
 '82009400-01',
 '10012100-01',
 '82010400-01',
 '10000200-01',
 '27004201-01',
 '82009002-01',
 '70002600-01',
 '82013700-01',
 '19002300-01',
 '82008900-01',
 '19002200-01',
 '82015900-01',
 '82010300-01',
 '82036800-01',
 '82011602-01',
 '82009200-01',
 '82005400-01',
 '19002700-01',
 '10001900-01',
 '19002900-01',
 '27005300-01',
 '82012300-01',
 '13005300-01',
 '27007000-01',
 '82007700-01',
 '19002400-01',
 '82009700-01',
 '19002100-01',
 '19044600-01',
 '27062700-01',
 '02000500-01',
 '82010100-01',
 '10005200-01',
 '10001100-01',
 '19003300-01',
 '19002601-01',
 '19034800-01',
 '82003400-01',
 '82012200-01',
 '82002000-01',
 '19003100-01',
 '19002500-01',
 '10009500-01',
 '27003501-01']

## Problem 4 - Create and write the final water quality table.

Finally, you should filter the table from **Problem 2.** to the lakes with complete information, then write this table to a parquet file named `water_quality_by_year.parquet`.

In [43]:
(water_quality_summaries_complete := water_quality_summaries
 .filter(pl.col('DNR_ID_Site_Number').is_in(dnr_id_site_numbers))
 )

DNR_ID_Site_Number,Year,LAKE_NAME,avg_secchi_depth,avg_total_phosphorus
str,i32,str,f64,f64
"""82015300-01""",2012,"""Sunset Lake""",3.164286,0.025429
"""27071100-01""",2010,"""Westwood Lake""",1.327778,0.022667
"""82033400-01""",2006,"""Kismet Lake""",1.840538,0.067769
"""82011301-01""",2005,"""Goose Lake""",0.972667,0.099905
"""82008700-01""",2009,"""Regional Park Lake""",2.460286,0.052286
…,…,…,…,…
"""82015300-01""",2007,"""Sunset Lake""",2.97,0.022
"""10005200-01""",2004,"""Reitz Lake""",1.936364,0.109091
"""10012100-01""",2005,"""Eagle Lake""",1.6555,0.1174


In [46]:
water_quality_summaries_complete.write_parquet('./data/water_quality_by_year.parquet')